In [6]:
import IPython.display as ipd
import torch
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
import re
from text.k2j import korean2katakana
from text.j2k import japanese2korean


# 일본어로 학습한 경우 True, 한국어로 학습한 경우 False로 isJaModel 값을 바꿔주세요.
isJaModel = False # True
model_name = 'ko' # ja
config_file = f"./configs/{model_name}.json"
model_file = f"./logs/{model_name}/G_91000.pth"
device = 'cpu' # cuda:0


text = """
[KO]안녕하세요.[KO]
[JA]こんにちは.[JA]
"""

text = re.sub('[\n]', '', text).strip()
if isJaModel:
    text = re.sub(r'\[KO\](.*?)\[KO\]', lambda x: korean2katakana(x.group(1)), text)
else:
    text = re.sub(r'\[JA\](.*?)\[JA\]', lambda x: japanese2korean(x.group(1)), text)

print(text)

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm


hps = utils.get_hparams_from_file(config_file)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint(model_file, net_g, None)

stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))

[KO]안녕하세요.[KO][PREPROCESSED]ㅋㅗㄴㄴㅣㅊㅣㅇㅗㅏ.[PREPROCESSED]
Mutli-stream iSTFT VITS
